# Apache Solr and Elasticsearch

Note: The RenderJSON() function in this Notebook is a 3rd party script, you can find here (along with an installation guide): https://www.reddit.com/r/IPython/comments/34t4m7/lpt_print_json_in_collapsible_format_in_ipython/

## Part I &mdash; Apache Solr

Main site: http://lucene.apache.org/solr/

Public Apache Solr examples: http://wiki.apache.org/solr/PublicServers

### Install Apache Solr

Requirement: JDK 6 or above

* Download from http://lucene.apache.org/solr/
* extract it: `tar -zxvf solr-5.3.1.tgz`
* start Solr:

```
$ cp path/to/solr-5.3.1
$ bin/solr -e techproducts
```
* try it: http://localhost:8983/solr/#/techproducts


In [3]:
import urllib.request
import json
import requests

def getContent(url):
  ufile = urllib.request.urlopen(url)
  return ufile.read().decode('utf8')
  
def getJson(url):
  text = json.loads(getContent(url))
  return text

def getEC(postdata):
  req = requests.post("http://localhost:9200/bank/_search", data = postdata)
  return req.json()

def printSource(postdata):
  data = getEC(postdata)
  for hit in data['hits']['hits']:
    print(hit['_source'])

Select all: `q=*:*`
where 1st `*` means every field, 2nd `*` means every value
```
/select?q=*:*&wt=json&indent=true
```

In [2]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=*%3A*&wt=json&indent=true"))

Get more: `rows=15` (~ LIMIT in SQL)
```
/select?q=*:*&rows=15
```

In [3]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=*%3A*&rows=15&wt=json"))

Paginate: `start=10` (~ OFFSET in SQL)
```
/select?q=*:*&start=10rows=10
```

In [4]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=*%3A*&start=10&rows=10&wt=json"))

Query a term: `q=usb`

In [5]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=usb&wt=json"))

Limit the fields to retrieve: `f=id,manu`

In [6]:
jsonObj = getJson("http://localhost:8983/solr/techproducts/select?q=usb&fl=id,manu&wt=json")
jsonObj['response']['docs']

[{'id': 'IW-02', 'manu': 'Belkin'},
 {'id': '3007WFP', 'manu': 'Dell, Inc.'},
 {'id': '9885A004', 'manu': 'Canon Inc.'},
 {'id': 'MA147LL/A', 'manu': 'Apple Computer Inc.'},
 {'id': '0579B002', 'manu': 'Canon Inc.'}]

Facets
===
* Enable faceting: `facet=true`
* Get list of manufactures: `facet.field=manu_exact`

In [7]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=usb&facet=true&facet.field=manu_exact&wt=json"))

* making it an association array in JSON response: `json.nl=map`

In [8]:
jsonObj = getJson("http://localhost:8983/solr/techproducts/select?q=usb&facet=true&facet.field=manu_exact&json.nl=map&wt=json")
jsonObj['facet_counts']['facet_fields']['manu_exact']

{'A-DATA Technology Inc.': 0,
 'ASUS Computer Inc.': 0,
 'ATI Technologies': 0,
 'Apache Software Foundation': 0,
 'Apple Computer Inc.': 1,
 'Bank of America': 0,
 'Bank of Norway': 0,
 'Belkin': 1,
 'Canon Inc.': 2,
 'Corsair Microsystems Inc.': 0,
 'Dell, Inc.': 1,
 'European Union': 0,
 'Maxtor Corp.': 0,
 'Samsung Electronics Co. Ltd.': 0,
 'U.K.': 0,
 'ViewSonic Corp.': 0}

* removing non matching values: `facet.mincount=1`

In [9]:
jsonObj = getJson("http://localhost:8983/solr/techproducts/select?q=usb&facet=true&facet.field=manu_exact&json.nl=map&facet.mincount=1&wt=json")
jsonObj['facet_counts']['facet_fields']['manu_exact']

{'Apple Computer Inc.': 1, 'Belkin': 1, 'Canon Inc.': 2, 'Dell, Inc.': 1}

* sort by count: `facet.sort=count`

In [10]:
jsonObj = getJson("http://localhost:8983/solr/techproducts/select?q=usb&facet=true&facet.field=manu_exact&json.nl=map&facet.mincount=1&facet.sort=count&wt=json")
RenderJSON(jsonObj)

Hit highlighting (why I get this response?)
===
* highlight on `hl=true`
* select field(s) to highight: `hl.fl=features`
* select HTML tag to highlight: `hl.simple.pre=<em>` and `hl.simple.post=</Fem>`
* query this time in the same field: `q=features:usb`

In [11]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=features:usb&fl=features&wt=json&indent=true&hl=true&hl.fl=features&hl.simple.pre=%3Cem%3E&hl.simple.post=%3C%2Fem%3E"))

Similar documents
===
The MoreLikeThis search component

* `mlt=true`        // MoreLikeThis search component is turned on
* `mlt.fl=manu,cat` // the fields to use in similarity
* `mlt.mindf=1`     // minimum document frequency (words must occur in at least this many docs)
* `mlt.mintf=1`     // minimum term frequency (the frequency below which terms will be ignored in the source doc)
* `fl=id,score`     // fields to return

In [4]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=usb&mlt=true&mlt.fl=manu,cat&mlt.mindf=1&mlt.mintf=1&fl=id,score&wt=json"))

Debugging (deep dive into term space vector)
===
* `debugQuery=true`                // turn on debugger
* `debug.explain.structured=true`  // return scores as structured data

In [5]:
RenderJSON(getJson("http://localhost:8983/solr/techproducts/select?q=usb%20OR%20disks&rows=3&debugQuery=true&debug.explain.structured=true&wt=json"))

## Part II &mdash; Elasticsearch

Main site: https://www.elastic.co/products/elasticsearch. Use cases: https://www.elastic.co/use-cases

This part is based on the Elasticsearch Getting Started Guide, you can find more explanation there: https://www.elastic.co/guide/en/elasticsearch/reference/2.1/getting-started.html

## Install Elasticsearch

Requirement: Java 7

1. Download Elasticsearch from https://www.elastic.co/downloads/elasticsearch
2. Uncompress
3. `bin/elasticsearch`
4. download a sample index (account.zip) from https://www.elastic.co/downloads/elasticsearch
5. unzip
6. `curl -XPOST 'localhost:9200/bank/account/_bulk?pretty' --data-binary @accounts.json`

Elasticseach provides a two-level index: `bank` (index) and `account` (type).

Search for everything: * (asterisk)

In [14]:
RenderJSON(getJson("http://localhost:9200/bank/_search?q=*"))

Match all - with a query language
```JSON
{
  "query": {"match_all": {}}
}
```

In [75]:
postdata = """
{
  "query": {"match_all": {}}
}"""
printSource(postdata)

{'account_number': 25, 'age': 39, 'gender': 'F', 'firstname': 'Virginia', 'balance': 40540, 'city': 'Nicholson', 'state': 'PA', 'email': 'virginiaayala@filodyne.com', 'employer': 'Filodyne', 'address': '171 Putnam Avenue', 'lastname': 'Ayala'}
{'account_number': 44, 'age': 37, 'gender': 'M', 'firstname': 'Aurelia', 'balance': 34487, 'city': 'Yardville', 'state': 'DE', 'email': 'aureliaharding@orbalix.com', 'employer': 'Orbalix', 'address': '502 Baycliff Terrace', 'lastname': 'Harding'}
{'account_number': 99, 'age': 39, 'gender': 'F', 'firstname': 'Ratliff', 'balance': 47159, 'city': 'Shaft', 'state': 'ND', 'email': 'ratliffheath@zappix.com', 'employer': 'Zappix', 'address': '806 Rockwell Place', 'lastname': 'Heath'}
{'account_number': 119, 'age': 28, 'gender': 'F', 'firstname': 'Laverne', 'balance': 49222, 'city': 'Herlong', 'state': 'DC', 'email': 'lavernejohnson@senmei.com', 'employer': 'Senmei', 'address': '302 Howard Place', 'lastname': 'Johnson'}
{'account_number': 126, 'age': 39,

Return only the first record with `size` (~ LIMIT)

In [74]:
postdata = """
{
  "query": {"match_all": {}},
  "size": 1
}"""
printSource(postdata)

{'account_number': 25, 'age': 39, 'gender': 'F', 'firstname': 'Virginia', 'balance': 40540, 'city': 'Nicholson', 'state': 'PA', 'email': 'virginiaayala@filodyne.com', 'employer': 'Filodyne', 'address': '171 Putnam Avenue', 'lastname': 'Ayala'}


Return records 10-20 with `from` (~ OFFSET) and `size` (~ LIMIT)

In [73]:
postdata = """
{
  "query": {"match_all": {}},
  "from": 10,
  "size": 10
}"""
printSource(postdata)

{'account_number': 227, 'age': 22, 'gender': 'M', 'firstname': 'Coleman', 'balance': 19780, 'city': 'Eagleville', 'state': 'WV', 'email': 'colemanberg@exoteric.com', 'employer': 'Exoteric', 'address': '776 Little Street', 'lastname': 'Berg'}
{'account_number': 253, 'age': 31, 'gender': 'M', 'firstname': 'Melissa', 'balance': 20240, 'city': 'Lumberton', 'state': 'MD', 'email': 'melissagould@buzzopia.com', 'employer': 'Buzzopia', 'address': '440 Fuller Place', 'lastname': 'Gould'}
{'account_number': 260, 'age': 30, 'gender': 'F', 'firstname': 'Kari', 'balance': 2726, 'city': 'Rushford', 'state': 'WV', 'email': 'kariskinner@singavera.com', 'employer': 'Singavera', 'address': '735 Losee Terrace', 'lastname': 'Skinner'}
{'account_number': 265, 'age': 26, 'gender': 'F', 'firstname': 'Marion', 'balance': 46910, 'city': 'Maplewood', 'state': 'WY', 'email': 'marionschneider@evidends.com', 'employer': 'Evidends', 'address': '574 Everett Avenue', 'lastname': 'Schneider'}
{'account_number': 335, '

Sorting hit list by `sort` (~ ORDER BY)

In [72]:
postdata = """
{
  "query": {"match_all": {}},
  "sort": { "balance": { "order": "desc" } }
}"""
printSource(postdata)

{'account_number': 248, 'age': 36, 'gender': 'M', 'firstname': 'West', 'balance': 49989, 'city': 'Maury', 'state': 'WA', 'email': 'westengland@obliq.com', 'employer': 'Obliq', 'address': '717 Hendrickson Place', 'lastname': 'England'}
{'account_number': 854, 'age': 25, 'gender': 'F', 'firstname': 'Jimenez', 'balance': 49795, 'city': 'Moscow', 'state': 'AL', 'email': 'jimenezbarry@verton.com', 'employer': 'Verton', 'address': '603 Cooper Street', 'lastname': 'Barry'}
{'account_number': 240, 'age': 35, 'gender': 'F', 'firstname': 'Oconnor', 'balance': 49741, 'city': 'Kilbourne', 'state': 'NH', 'email': 'oconnorclay@franscene.com', 'employer': 'Franscene', 'address': '659 Highland Boulevard', 'lastname': 'Clay'}
{'account_number': 97, 'age': 40, 'gender': 'F', 'firstname': 'Karen', 'balance': 49671, 'city': 'Fredericktown', 'state': 'MO', 'email': 'karentrujillo@tsunamia.com', 'employer': 'Tsunamia', 'address': '512 Cumberland Walk', 'lastname': 'Trujillo'}
{'account_number': 842, 'age': 

Select fields (~ SELECT statement)

In [71]:
postdata = """
{
  "query": {"match_all": {}},
  "_source": ["account_number", "balance"]
}"""
printSource(postdata)

{'account_number': 25, 'balance': 40540}
{'account_number': 44, 'balance': 34487}
{'account_number': 99, 'balance': 47159}
{'account_number': 119, 'balance': 49222}
{'account_number': 126, 'balance': 3607}
{'account_number': 145, 'balance': 47406}
{'account_number': 183, 'balance': 14223}
{'account_number': 190, 'balance': 3150}
{'account_number': 208, 'balance': 40760}
{'account_number': 222, 'balance': 14764}


Query by exact numeric field value with `match`

In [70]:
postdata = """
{
  "query": { "match": { "account_number": 20 } }
}"""
printSource(postdata)

{'account_number': 20, 'age': 36, 'gender': 'M', 'firstname': 'Elinor', 'balance': 16418, 'city': 'Ribera', 'state': 'WA', 'email': 'elinorratliff@scentric.com', 'employer': 'Scentric', 'address': '282 Kings Place', 'lastname': 'Ratliff'}


Query by text field values

In [69]:
postdata = """
{
  "query": { "match": { "address": "mill" } }
}"""
printSource(postdata)

{'account_number': 472, 'age': 32, 'gender': 'F', 'firstname': 'Lee', 'balance': 25571, 'city': 'Movico', 'state': 'MT', 'email': 'leelong@comverges.com', 'employer': 'Comverges', 'address': '288 Mill Street', 'lastname': 'Long'}
{'account_number': 136, 'age': 38, 'gender': 'M', 'firstname': 'Winnie', 'balance': 45801, 'city': 'Urie', 'state': 'IL', 'email': 'winnieholland@neteria.com', 'employer': 'Neteria', 'address': '198 Mill Lane', 'lastname': 'Holland'}
{'account_number': 970, 'age': 28, 'gender': 'M', 'firstname': 'Forbes', 'balance': 19648, 'city': 'Lopezo', 'state': 'AK', 'email': 'forbeswallace@pheast.com', 'employer': 'Pheast', 'address': '990 Mill Road', 'lastname': 'Wallace'}
{'account_number': 345, 'age': 38, 'gender': 'M', 'firstname': 'Parker', 'balance': 9812, 'city': 'Blackgum', 'state': 'KY', 'email': 'parkerhines@baluba.com', 'employer': 'Baluba', 'address': '715 Mill Avenue', 'lastname': 'Hines'}


mill OR lane

In [68]:
postdata = """
{
  "query": { "match": { "address": "mill lane" } },
  "_source": ["address"]
}"""
printSource(postdata)

{'address': '198 Mill Lane'}
{'address': '288 Mill Street'}
{'address': '990 Mill Road'}
{'address': '715 Mill Avenue'}
{'address': '929 Eldert Lane'}
{'address': '410 Jamison Lane'}
{'address': '591 Nolans Lane'}
{'address': '448 Cove Lane'}
{'address': '945 Bedell Lane'}
{'address': '317 Pooles Lane'}


"mill lane" as one phrase with `match_prase`

In [67]:
postdata = """
{
  "query": { "match_phrase": { "address": "mill lane" } },
  "_source": ["address"]
}"""
printSource(postdata)

{'address': '198 Mill Lane'}


Boolean AND: mill AND lane with `bool`, `must` and `match`

In [66]:
postdata = """
{
  "query": {
    "bool": {
      "must": [
        { "match": { "address": "mill" } },
        { "match": { "address": "lane" } }
      ]
    }
  },
  "_source": ["address"]
}"""
printSource(postdata)

{'address': '198 Mill Lane'}


Boolean OR: mill OR lane with `bool`, `should` and `match`

In [65]:
postdata = """
{
  "query": {
    "bool": {
      "should": [
        { "match": { "address": "mill" } },
        { "match": { "address": "lane" } }
      ]
    }
  },
  "_source": ["address"]
}"""
printSource(postdata)

{'address': '198 Mill Lane'}
{'address': '288 Mill Street'}
{'address': '990 Mill Road'}
{'address': '715 Mill Avenue'}
{'address': '929 Eldert Lane'}
{'address': '410 Jamison Lane'}
{'address': '591 Nolans Lane'}
{'address': '448 Cove Lane'}
{'address': '945 Bedell Lane'}
{'address': '317 Pooles Lane'}


Boolean NOT: neither "mill" nor "lane" with `must_not`

In [64]:
postdata = """
{
  "query": {
    "bool": {
      "must_not": [
        { "match": { "address": "mill" } },
        { "match": { "address": "lane" } }
      ]
    }
  },
  "_source": ["address"]
}"""
printSource(postdata)

{'address': '171 Putnam Avenue'}
{'address': '502 Baycliff Terrace'}
{'address': '806 Rockwell Place'}
{'address': '302 Howard Place'}
{'address': '620 National Drive'}
{'address': '891 Elton Street'}
{'address': '823 Herkimer Place'}
{'address': '636 Diamond Street'}
{'address': '810 Nostrand Avenue'}
{'address': '333 Narrows Avenue'}


A more complex example: `age` = 40 but `state` is not Indiana

In [63]:
postdata = """
{
  "query": {
    "bool": {
      "must": [
        { "match": { "age": "40" } }
      ],
      "must_not": [
        { "match": { "state": "ID" } }
      ]
    }
  },
  "_source": ["age", "state"]
}"""
printSource(postdata)

{'age': 40, 'state': 'OR'}
{'age': 40, 'state': 'CA'}
{'age': 40, 'state': 'NC'}
{'age': 40, 'state': 'KY'}
{'age': 40, 'state': 'AL'}
{'age': 40, 'state': 'HI'}
{'age': 40, 'state': 'VT'}
{'age': 40, 'state': 'AK'}
{'age': 40, 'state': 'OR'}
{'age': 40, 'state': 'WA'}


### Filters
Purpose: reuse query cache 

In [77]:
postdata = """
{
  "query": {
    "bool": {
      "must": { "match_all": {} },
      "filter": {
        "range": {
          "balance": {
            "gte": 20000,
            "lte": 30000
          }
        }
      }
    }
  },
  "_source": ["age", "state", "balance"]
}"""
printSource(postdata)

{'age': 31, 'balance': 20240, 'state': 'MD'}
{'age': 21, 'balance': 20685, 'state': 'VT'}
{'age': 32, 'balance': 27987, 'state': 'OR'}
{'age': 26, 'balance': 29362, 'state': 'NE'}
{'age': 23, 'balance': 20325, 'state': 'PA'}
{'age': 21, 'balance': 25291, 'state': 'AL'}
{'age': 35, 'balance': 22575, 'state': 'OK'}
{'age': 39, 'balance': 20480, 'state': 'NY'}
{'age': 28, 'balance': 27894, 'state': 'KS'}
{'age': 39, 'balance': 27714, 'state': 'LA'}


### Aggregations

In [83]:
postdata = """
{
  "size": 0,
  "aggs": {
    "group_by_state": {
      "terms": {
        "field": "state"
      }
    }
  }
}"""
data = getEC(postdata)
RenderJSON(data)
for item in data['aggregations']['group_by_state']['buckets']:
  print(item)

{'doc_count': 30, 'key': 'tx'}
{'doc_count': 28, 'key': 'md'}
{'doc_count': 27, 'key': 'id'}
{'doc_count': 25, 'key': 'al'}
{'doc_count': 25, 'key': 'me'}
{'doc_count': 25, 'key': 'tn'}
{'doc_count': 25, 'key': 'wy'}
{'doc_count': 24, 'key': 'dc'}
{'doc_count': 24, 'key': 'ma'}
{'doc_count': 24, 'key': 'nd'}


adding more info to aggregation

In [86]:
postdata = """
{
  "size": 0,
  "aggs": {
    "group_by_state": {
      "terms": {
        "field": "state"
      },
      "aggs": {
        "average_balance": {
          "avg": {
            "field": "balance"
          }
        }
      }
    }
  }
}"""
data = getEC(postdata)
RenderJSON(data)
for item in data['aggregations']['group_by_state']['buckets']:
  print(item)

{'doc_count': 30, 'key': 'tx', 'average_balance': {'value': 26073.3}}
{'doc_count': 28, 'key': 'md', 'average_balance': {'value': 26161.535714285714}}
{'doc_count': 27, 'key': 'id', 'average_balance': {'value': 24368.777777777777}}
{'doc_count': 25, 'key': 'al', 'average_balance': {'value': 25739.56}}
{'doc_count': 25, 'key': 'me', 'average_balance': {'value': 21663.0}}
{'doc_count': 25, 'key': 'tn', 'average_balance': {'value': 28365.4}}
{'doc_count': 25, 'key': 'wy', 'average_balance': {'value': 21731.52}}
{'doc_count': 24, 'key': 'dc', 'average_balance': {'value': 23180.583333333332}}
{'doc_count': 24, 'key': 'ma', 'average_balance': {'value': 29600.333333333332}}
{'doc_count': 24, 'key': 'nd', 'average_balance': {'value': 26577.333333333332}}


order by the extra info

In [87]:
postdata = """
{
  "size": 0,
  "aggs": {
    "group_by_state": {
      "terms": {
        "field": "state",
        "order": {
          "average_balance": "desc"
        }
      },
      "aggs": {
        "average_balance": {
          "avg": {
            "field": "balance"
          }
        }
      }
    }
  }
}"""
data = getEC(postdata)
RenderJSON(data)
for item in data['aggregations']['group_by_state']['buckets']:
  print(item)

{'doc_count': 14, 'key': 'co', 'average_balance': {'value': 32460.35714285714}}
{'doc_count': 16, 'key': 'ne', 'average_balance': {'value': 32041.5625}}
{'doc_count': 14, 'key': 'az', 'average_balance': {'value': 31634.785714285714}}
{'doc_count': 17, 'key': 'mt', 'average_balance': {'value': 31147.41176470588}}
{'doc_count': 16, 'key': 'va', 'average_balance': {'value': 30600.0625}}
{'doc_count': 19, 'key': 'ga', 'average_balance': {'value': 30089.0}}
{'doc_count': 24, 'key': 'ma', 'average_balance': {'value': 29600.333333333332}}
{'doc_count': 22, 'key': 'il', 'average_balance': {'value': 29489.727272727272}}
{'doc_count': 14, 'key': 'nm', 'average_balance': {'value': 28792.64285714286}}
{'doc_count': 17, 'key': 'la', 'average_balance': {'value': 28791.823529411766}}


In [91]:
postdata = """
{
  "size": 0,
  "aggs": {
    "group_by_age": {
      "range": {
        "field": "age",
        "ranges": [
          {"from": 20, "to": 30},
          {"from": 30, "to": 40},
          {"from": 40, "to": 50}
        ]
      },
      "aggs": {
        "group_by_gender": {
          "terms": {
            "field": "gender"
          },
          "aggs": {
            "average_balance": {
              "avg": {
                "field": "balance"
              }
            }
          }
        }
      }
    }
  }
}"""
data = getEC(postdata)
RenderJSON(data)
#for item in data['aggregations']['group_by_age']['buckets']:
#  print(item)